# Machine Learning Workshop

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from visualization import *

## Generate data

In [ ]:
# Set seed for reproducibility
SEED = 1

# Generate a linear regression dataset
X, y = datasets.make_regression(
    n_samples=100, 
    n_features=1, 
    noise=10,
    random_state=SEED,
    bias=100.0
)
X = X.flatten() * 5 # Makes this example more interesting

# Display first few samples of the dataset
print("First 5 samples of the dataset:")
for i in range(5):
    print(f"X: {X[i]:.2f}, y: {y[i]:.2f}")

## Split the data into train, validation, and test sets

In [ ]:
# Split the data into train, validation, and test sets (60%, 20%, 20%)
X_train, X_temp, y_train, y_temp = None, None, None, None   # TODO
X_val, X_test, y_val, y_test = None, None, None, None       # TODO

# Scale the data using mean and standard deviation and using only training data statistics
# X_mean = np.mean(X_train)
# X_std = np.std(X_train)
# X_train = (X_train - X_mean) / X_std
# X_val = (X_val - X_mean) / X_std
# X_test = (X_test - X_mean) / X_std

# y_mean = np.mean(y_train)
# y_std = np.std(y_train)
# y_train = (y_train - y_mean) / y_std
# y_val = (y_val - y_mean) / y_std
# y_test = (y_test - y_mean) / y_std


plot_train_val_data(X_train, y_train, X_val, y_val)

## Functions for linear regression


In [5]:
def predict(X: np.ndarray, weights: np.ndarray) -> np.ndarray:
    """Compute predictions using linear regression"""
    
    ... # TODO


def compute_loss_and_gradient(X: np.ndarray, y: np.ndarray, weights: np.ndarray) -> tuple[float, np.ndarray]:
    """
    Computes both MSE loss and its gradient in a single pass through the data
    
    Returns:
        tuple: (loss, gradient)
    """

    ... # TODO


def gradient_descent(
    X: np.ndarray, 
    y: np.ndarray,
    weights: np.ndarray=np.array([0, 0]),
    learning_rate: float=0.1,
    iterations: int=1000,
) -> np.ndarray:
    """
    Iteratively updates the weights using the steepest descent method
    """
    
    ... # TODO


## Run gradient descent

In [ ]:
weights = None # TODO

print(f"Found weights: {weights}")

plot_linear_regression(X_train, y_train, weights, title="Linear Regression - Training Data")

In [ ]:
plot_linear_regression(X_val, y_val, weights, title="Linear Regression - Validation Data", scatter_color="orange")


## Run hyperparameter search

In [ ]:
learning_rates = [] # TODO
iterations = []     # TODO

best_learning_rate = None
best_iterations = None
best_weights = None
best_mse_val_loss = float('inf')

plt.validation_results = {'lr': [], 'iterations': [], 'mse': []}

for learning_rate in learning_rates:
    for iteration in iterations:
        weights = None                  # TODO
        mse_val_loss, _ = None, None    # TODO
        
        # Store results for plotting
        plt.validation_results['lr'].append(learning_rate)
        plt.validation_results['iterations'].append(iteration)
        plt.validation_results['mse'].append(mse_val_loss)
        
        print(f"Learning rate = {learning_rate:.5f} | Iterations = {iteration}\tMSE on validation set: {mse_val_loss}")
        
        # Update best hyperparameters if current validation loss is lower
        ... # TODO


plot_validation_results(learning_rates, plt.validation_results)

In [ ]:
print(f"\nBest learning rate: {best_learning_rate:.5f}")
print(f"Best iterations: {best_iterations}")
print(f"Best MSE on validation set: {best_mse_val_loss}")

plot_linear_regression(X_val, y_val, best_weights, title="Linear Regression - Validation Data", scatter_color="orange")

In [ ]:
plot_regression_progression(X_train, y_train, X_val, y_val, gradient_descent)


## Evaluate on test set

In [ ]:
# Combine train and validation sets
X_combined = None # TODO
y_combined = None # TODO

# Train the model on combined data with best hyperparameters
weights = None # TODO

# Evaluate on test set
mse_test_loss, _ = None, None # TODO
print(f"MSE on test set: {mse_test_loss}")


plot_linear_regression(X_test, y_test, weights, title="Linear Regression - Test Data", scatter_color="red")

## Visualize gradient descent on loss surface

In [ ]:
plot_loss_surface(X, y, compute_loss_and_gradient, dim='2d', lr=best_learning_rate, n_iter=best_iterations, init_weights=[0, 0])


In [ ]:
plot_loss_surface(X, y, compute_loss_and_gradient, dim='3d', lr=best_learning_rate, n_iter=best_iterations, init_weights=[0, 0])

In [ ]:
plot_loss_surface(X, y, compute_loss_and_gradient, dim='3d', lr=0.005, n_iter=500, init_weights=[-150, 110])

In [ ]:
plot_loss_surface(X, y, compute_loss_and_gradient, dim='3d', lr=0.05, n_iter=100, init_weights=[-150, 110])